In [1]:
import pandas as pd
import json
import re
import csv

In [2]:
expected_tempest_filename = 'expected-tempest-spells.csv'
expected_base_filename = 'expected-base-spells.csv'

input_filename = 'hardcodex-cleric-all.csv'
col_names = ['level', 'title', 'type', 'casting_time', 'range', 'components', 'duration', 'orig_description', 'orig_class']

output_filename = 'cleric-tempest-5.json'

overrides_filename = 'overrides.txt'


subclass_reg = re.compile('(\w+)\s+\((\w+)\)')
def parse_class(input_str):
    m = subclass_reg.match(input_str)
    if m:
        data = m.groups()
    else:
        data = (input_str, None)

    return pd.Series(data=data)

ritual_reg = re.compile('(.+)\s+\([Rr]itual\)')
def parse_ritual(input_str):
    m = ritual_reg.match(input_str)
    if m:
        data = (m.group(1), True)
    else:
        data = (input_str, False)

    return pd.Series(data=data)


orig_df = pd.read_csv(input_filename, sep=';', header=None, names=col_names)
orig_df[['klass','subclass']] = orig_df['orig_class'].apply(parse_class)
orig_df[['title', 'is_ritual']] = orig_df['title'].apply(parse_ritual)

orig_expected = pd.read_csv(expected_base_filename, sep=',', header=None, names=['level', 'title', 'subclass'])
orig_expected['subclass'] = orig_expected['subclass'].apply(lambda x: None if pd.isnull(x) else x)

orig_expected_tempest = pd.read_csv(expected_tempest_filename, sep=',', header=None, names=['level', 'title', 'subclass'])

# format data
orig_df = orig_df.set_index(['title'], drop=False)

orig_expected = orig_expected.set_index(['title'], drop=False)

orig_expected_tempest = orig_expected_tempest.set_index(['title'], drop=False)

In [3]:
# Calculate expected cards
df = orig_df.copy()
expected = orig_expected.copy()
expected_tempest = orig_expected_tempest.copy()

# combine expected
all_expected = expected.append(expected_tempest)
all_expected = all_expected.groupby(['title', 'level'])['subclass'].apply(lambda x: next((a for a in x if a), None)).reset_index()
all_expected = all_expected[all_expected['level'] < 5]
all_expected = all_expected.sort_values(by=['level', 'title'])
all_expected = all_expected.set_index(['title'], drop=False)
lvl_4 = all_expected[all_expected['level'] == 4]

assert len(lvl_4) == 9

In [4]:
#print(df[df['title'].isin(['Augury', 'Bless', 'Control Water'])]['subclass'])
wtf = pd.merge(df, all_expected, on=['level', 'title'], left_index=True, right_index=True)
wtf = wtf.drop('subclass_x', axis=1)
wtf = wtf.rename(columns={'subclass_y': 'subclass'})

wtf = wtf[(wtf.klass == 'Cleric') & wtf.subclass.isin(['Tempest', None]) & (wtf.level <= 4)]

assert len(wtf) == len(all_expected), 'Expected %d but got %d' % (len(all_expected), len(wtf))

wtf = wtf.sort_values(by=['level', 'title'])

flavor_reg = re.compile('\(([\w ]+)\)(.*)')    
def parse_flavor(desc):
    m = flavor_reg.match(desc)
    if m:
        data = m.groups()
    else:
        data = (None, desc)

    return pd.Series(data=data)

at_higher_reg = re.compile('(.*)At Higher Levels:\s+?(.*)')    
def parse_at_higher(desc):
    m = at_higher_reg.match(desc)
    if m:
        data = m.groups()
    else:
        data = (desc, None)

    return pd.Series(data=data)


wtf[['flavor', 'description']] = wtf['orig_description'].apply(parse_flavor)
wtf[['description', 'at_higher']] = wtf['description'].apply(parse_at_higher)

spells = wtf

In [5]:
with open(overrides_filename) as fp:
    overrides = {}
    current = None
    destination = 'description'
    for line in fp:
        line = line.strip()
        if not line:
            continue
        if line.find('----at higher levels') == 0:
            destination = 'at_higher'
            continue

        if line.find('---') == 0:
            title = line[3:]
            overrides[title] = current = {'description': '', 'at_higher': ''}
            destination = 'description'
            continue
            
        current[destination] += line
            

data = []
for idx, spell in spells.iterrows():
    if spell.subclass:
        title = '{0} ({1})'.format(spell.title, spell.subclass)
    else:
        title = spell.title
    contents = []
    contents.append('subtitle | {}'.format(spell.type))
    if False:
        contents.append('property | Casting time | {}'.format(spell.casting_time))
        contents.append('property | Range | {}'.format(spell.range))
        if False:  # include components?
            contents.append('property | Components | {}'.format(spell.components))
        contents.append('property | Duration | {}'.format(spell.duration))

    else:
        components = ','.join(map(lambda x: x.strip(), spell.components.split(',')))
        casting_time = spell.casting_time
        duration = spell.duration.replace('Concentration,', 'CONC')
        combined_stats = '({}) {}, {}'.format(components, casting_time, duration)
        contents.append('text | ' + combined_stats)
        contents.append('property | Range | {}'.format(spell.range))

    contents.append('rule')

    override_description = overrides.get(spell.title, {}).get('description')
    description_chunks = (override_description or spell.description).split('<br>')


    for chunk in description_chunks:
        chunk = chunk.strip()

        # get rid of component details that show up sometimes
        if chunk.find('(') == 0:
            chunk = chunk[chunk.find(')')+1:]

        if not chunk:
            continue

        contents.append('text | {}'.format(chunk))

    override_at_higher = overrides.get(spell.title, {}).get('at_higher')
    at_higher = override_at_higher or spell.at_higher
    if at_higher:
        at_higher_chunks = at_higher.split('<br>')
        at_higher_chunks = map(lambda x: x.strip(), at_higher_chunks)
        if at_higher_chunks:
            contents.append('rule')
            for chunk in at_higher_chunks:
                chunk = chunk.strip()
                if not chunk:
                    continue
                contents.append('text | {}'.format(chunk))
    result = {
        'count': 1,
        'color': '',
        'icon_back': '',
        'icon': '',
        'title': title,
        'contents': contents,
    }
    data.append(result)
with open(output_filename, 'w') as fd:
    json.dump(data, fd, indent=4)

In [6]:
set(all_expected['title']) - set(spells['title'])

set()